# **Exercise 2. Autoencoder**

Second exercise for the Project of DLAI Fall semester 2019/2020.

Authors:
* Carlos Alejandro López Molina
* María González i Calabuig
* Roser Batlle Roca
* Jordi Biosca Caro

Import of different libraries used to build the script:

In [0]:
import torch

import numpy as np
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torchvision.utils import save_image
from torchvision import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data.sampler import SubsetRandomSampler
from tabulate import tabulate
from torch.utils import data 

seed = 1
torch.manual_seed(seed)
np.random.seed(seed)

Definition of hyper-parameters:

In [0]:
hparams = {
    'batch_size': 64,
    'num_epochs': 5,
    'test_batch_size': 64,
    'hidden_size': 64,
    'hidden_size_2': 128,
    'hidden_size_3': 64,
    'num_classes': 10,
    'num_inputs': 1, 
    'learning_rate': 1e-3,
    'log_interval': 100,
    'num_workers': 4,
    'kernel_size': 3,
    'encoder_key': [1, 5, 25, 50, 100],
    'subset_size': 100
}

hparams['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

**STEP 1** - Select 100 images from MNIST dataset

In [0]:
mnist_trainset = datasets.MNIST('data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
mnist_testset = datasets.MNIST('data', train=False, 
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))


train_loader = torch.utils.data.DataLoader(mnist_trainset, 
                                           hparams['batch_size'], 
                                           shuffle=False,
                                           num_workers=hparams['num_workers'], 
                                           )

test_loader = torch.utils.data.DataLoader(mnist_trainset, 
                                          hparams['batch_size'], 
                                          shuffle=False,
                                          num_workers=hparams['num_workers'], 
                                          )

samples_train = torch.utils.data.Subset(train_loader, 100)
samples_test = torch.utils.data.Subset(test_loader, 100)



**STEP 2** - Reusing the Encoder of the Autoencoder from Ex1. 
Bottleneck size = 100. 

In [0]:
class ConvAutoEncoder(nn.Module):

  def __init__(self):
    super(ConvAutoEncoder,self).__init__()

    self.encoder = nn.Sequential(
        nn.Conv2d(hparams['num_inputs'], hparams['hidden_size'], 4, padding=3, stride=2), # surten 16,16
        nn.ReLU(True),
        nn.Conv2d(hparams['hidden_size'], hparams['hidden_size_2'], 4, padding=1, stride=2), #surten 8,8
        nn.ReLU(True),
        nn.Conv2d(hparams['hidden_size_2'], 25, 4, stride=2), #surten 3,3
        nn.ReLU(True),
        nn.MaxPool2d(2, stride=1) # surten 2,2
    )

    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(25, hparams['hidden_size_2'], 3, stride=2), #surten 5,5
        nn.ReLU(True),
        nn.ConvTranspose2d(hparams['hidden_size_2'], hparams['hidden_size'], 5, stride=3, padding=1), # surten 15,15
        nn.ReLU(True),
        nn.ConvTranspose2d(hparams['hidden_size'], hparams['num_inputs'], 2, stride=2, padding=1), #surten 28,28
        nn.Tanh(),
    )

  def forward(self,x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x

In [0]:
losses_train = []
losses_test = []
model = ConvAutoEncoder()
model = model.to(hparams['device'])
distance = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['learning_rate'], weight_decay=1e-5)
for epoch in range (hparams['num_epochs']):
  for batch_id, data in enumerate(train_loader):
    img, _ = data
    img = img.to(hparams['device'])
    # Forward pass
    output = model(img)
    loss = distance(output, img)
    losses_train.append(loss.cpu().data.item())
    # Backward pass
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()
    # Log message
    print('TRAIN: epoch[{}/{}], loss:{:.4f}'.format(epoch+1, hparams['num_epochs'], loss.item()))
  for data in test_loader:
    img, _ = data
    img = img.to(hparams['device'])
    # Forward pass
    output = model(img)
    loss = distance(output, img)
    losses_test.append(loss.cpu().data.item())
    # Backward pass
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()
   # Log message
    print('TEST: epoch[{}/{}], loss:{:.4f}'.format(epoch+1, hparams['num_epochs'], loss.item()))

TRAIN: epoch[1/5], loss:0.3309
TRAIN: epoch[1/5], loss:0.3301
TRAIN: epoch[1/5], loss:0.3134
TRAIN: epoch[1/5], loss:0.3287
TRAIN: epoch[1/5], loss:0.2917
TRAIN: epoch[1/5], loss:0.2915
TRAIN: epoch[1/5], loss:0.2766
TRAIN: epoch[1/5], loss:0.2789
TRAIN: epoch[1/5], loss:0.2800
TRAIN: epoch[1/5], loss:0.2598
TRAIN: epoch[1/5], loss:0.2777
TRAIN: epoch[1/5], loss:0.2968
TRAIN: epoch[1/5], loss:0.2779
TRAIN: epoch[1/5], loss:0.2842
TRAIN: epoch[1/5], loss:0.2733
TRAIN: epoch[1/5], loss:0.2606
TRAIN: epoch[1/5], loss:0.2388
TRAIN: epoch[1/5], loss:0.2442
TRAIN: epoch[1/5], loss:0.2764
TRAIN: epoch[1/5], loss:0.2994
TRAIN: epoch[1/5], loss:0.2860
TRAIN: epoch[1/5], loss:0.3031
TRAIN: epoch[1/5], loss:0.2637
TRAIN: epoch[1/5], loss:0.2513
TRAIN: epoch[1/5], loss:0.2508
TRAIN: epoch[1/5], loss:0.2474
TRAIN: epoch[1/5], loss:0.2421
TRAIN: epoch[1/5], loss:0.2489
TRAIN: epoch[1/5], loss:0.2416
TRAIN: epoch[1/5], loss:0.2561
TRAIN: epoch[1/5], loss:0.2635
TRAIN: epoch[1/5], loss:0.2654
TRAIN: e

Saving the weights of the autoencoder:

In [0]:
pretrained_dict = model.state_dict()
torch.save(pretrained_dict, 'weights_autoencoder.pt')

**STEP 3** - Digit Classifier

In [0]:
class Classifier(nn.Module):

  def __init__(self):
    super(Classifier,self).__init__()

    self.encoder = nn.Sequential(
      nn.Conv2d(hparams['num_inputs'], hparams['hidden_size'], 4, padding=3, stride=2), # surten 16,16
      nn.ReLU(True),
      nn.Conv2d(hparams['hidden_size'], hparams['hidden_size_2'], 4, padding=1, stride=2), #surten 8,8
      nn.ReLU(True),
      nn.Conv2d(hparams['hidden_size_2'], 25, 4, stride=2), #surten 3,3
      nn.ReLU(True),
      nn.MaxPool2d(2, stride=1) # surten 2,2
    )

    self.fc = nn.Sequential(
        nn.Linear(100,10),     # El profe va comentar que Softmax podria ser una possibilitat també!!   
        nn.Softmax(dim=1)
    )
  
  def forward(self,x):
    x = self.encoder(x)
    x = x.view(-1)
    x = self.fc(x)
    return x


**STEP 4** - Pre-Training. 

In [0]:
model = Classifier()
model_dict = model.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
model = model.to(hparams['device'])

# model = Classifier()
# model_dict = model.state_dict()
# pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# model_dict.update(pretrained_dict)
# model = model.load_state_dict(torch.load('weights_autoencoder.pt'))
# model = model.to(hparams['device'])

distance = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['learning_rate'], weight_decay=1e-5)

for epoch in range (hparams['num_epochs']):
  for batch_id, data in enumerate(samples_train):
    img, _ = data
    img = img.to(hparams['device'])
    # Forward pass
    output = model(img)
    loss = distance(output, img)
    losses_train.append(loss.cpu().data.item())
    # Backward pass
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()
    # Log message
    print('TRAIN: epoch[{}/{}], loss:{:.4f}'.format(epoch+1, hparams['num_epochs'], loss.item()))
  for data in samples_test:
    img, _ = data
    img = img.to(hparams['device'])
    # Forward pass
    output = model(img)
    loss = distance(output, img)
    losses_test.append(loss.cpu().data.item())
    # Backward pass
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()
    # Log message
    print('TEST: epoch[{}/{}], loss:{:.4f}'.format(epoch+1, hparams['num_epochs'], loss.item()))

TypeError: ignored